In [1]:
import os
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.append(f"{project_root}/src")
sys.path.append(f"{project_root}/third_party")

from config import gpt2_cfg as cfg


In [2]:
import ray

if not ray.is_initialized():
    ray.init(
        runtime_env={
            "env_vars": {          
                "PYTHONPATH": "$PYTHONPATH:" + cfg.project_root + "/src",
            },
            "working_dir": cfg.project_root,
            "excludes": [
                "/bazel-*",
                ".git",
                "*.pyc",
                "/__pycache__",
                "/output",
                "/model",
            ],
           
        },
        _metrics_export_port=8080,
    )
# convience for debugging
ray.data.DataContext.get_current().execution_options.verbose_progress = True
ray.data.DataContext.log_internal_stack_trace_to_stdout = True

2024-07-29 12:03:43,571	WARNING services.py:2017 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.30gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-07-29 12:03:43,685	INFO worker.py:1781 -- Started a local Ray instance.
2024-07-29 12:03:43,690	INFO packaging.py:530 -- Creating a file package for local directory '/workspaces/CaiZi'.
2024-07-29 12:03:43,695	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_8d1dfbd98c4afcca.zip' (0.28MiB) to Ray cluster...
2024-07-29 12:03:43,696	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_8d1dfbd98c4afcca.zip'.


In [ ]:
from pathlib import Path
data_sources = [ Path(item["path"]) for item in cfg["dataset"]]
text_document_paths = ray.data.from_items(data_sources)

In [ ]:
from document_processor import TextDocumentProcessor
text_document_processor = TextDocumentProcessor()
texts=text_document_paths.map(text_document_processor)

In [ ]:
from token_processor import TikTokenizer
tokenizer = TikTokenizer()
tokens = texts.map(tokenizer)

In [ ]:
from chunk_processor import ChunkProcessor
chunk_processor = ChunkProcessor()
chunked_tokens = tokens.flat_map(chunk_processor)

In [ ]:
for batch in chunked_tokens.iter_torch_batches(batch_size=cfg.batch_size):
    print(batch)